In [5]:
#! python3
import docx
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import string
import nltk
import warnings
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)
#Text
#text = getText('7.11 Register of Known Risks Deliverable (5-20-2021 Martin updates) (Connor McSweeney).docx')
#Start Tom request
#7.10 OD Change Management Education Materials.docx
#NIH-DMI Plan for Implementing High-Level Cloud Governance Principles for ORS.docx
#DMI 7.10 Change Management Strategy_NIH OD Cyber Modernization_03-29-2021 final edits.docx
#Plan and Strategy for Implementing NIST 800-82 Controls Across ORSORF Rev_01.docx
#1
#text = getText('7.10 OD Change Management Education Materials.docx')
#2
train = pd.read_csv('ElonMuskTimeline2019through2021.csv')

combi = train


def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)

    return input_txt
pattern = r'(http://[^"\s]+)|(@\w+)'
combi['tweet'] = combi['tweet'].fillna('').apply(str)
combi['tweet'] = combi['tweet'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)
#combi['tweet'] = combi['tweet'].fillna('').apply(str)
combi['tidy_tweet'] = np.vectorize(remove_pattern)(combi['tweet'], "@[\w]*")
#combi['tidy_tweet']=combi['tidy_tweet'].apply(str)

## Now Correlation finding
!pip install --upgrade gensim
from gensim.models import Word2Vec, FastText
import pandas as pd
import re
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt

import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Creating Corpous and Vector
combi = combi[~combi['tweet'].str.contains("https")]
combi['tidy_tweet']=combi['tidy_tweet'].apply(str)
#combi['tidy_tweet']=combi['tidy_tweet'].dropna(inplace=True)
dfValaSentences = '\n'.join(combi["tidy_tweet"].astype(str).values)
 
#dfValaSentences['hashtag'] = dfValaSentences['tweet'].apply(lambda x: re.findall(r'\B#\w*[a-zA-Z]+\w*', x)) #creating a new colu

#tokens = df['label'].apply(word_tokenize)
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True)

from nltk import sent_tokenize
import nltk
#nltk.download()
nltk.download('punkt')
#print(dfValaSentences)

text = dfValaSentences[0:5000]


sentences = sent_tokenize(text)
#print(sentences[0])
#Removes clutter
tokenized_sent_after_remove_n = [x.replace('\n',' ') for x in sentences]
#Removes clutter
tokenized_sent_after_remove_n_and_t = [x.replace('\t',' ') for x in tokenized_sent_after_remove_n]
#Setences ready
sentences = tokenized_sent_after_remove_n_and_t
#print(tokenized_sent_after_remove_n_and_t)
print(sentences)
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 #Get the vector's for the words in the sentence
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
#Get the all the vectors for all the words
    all_words = list(set(sent1 + sent2))
#Set the
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 #Use Math Of Cosine to return the similiary between the vectors 
    return 1 - cosine_distance(vector1, vector2)
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    return similarity_matrix
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
top_n=20
stop_words = stopwords.words('english')
summarize_text = []
print('1')
# Step 2 - Generate Similary Martix across sentences
sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)
# Step 3 - Rank sentences in similarity martix
print('2')
sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
print('3')
scores = nx.pagerank(sentence_similarity_graph)
# Step 4 - Sort the rank and pick top sentences
print('4')
ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
print("Indexes of top ranked_sentence order are ", ranked_sentence)
for i in range(top_n):
    summarize_text.append((ranked_sentence[i][1]))
    # Step 5 - Offcourse, output the summarize texr
    print('OUTPUT SUMMARIZE TEXT')
    print(summarize_text)
print('5')
#print(combi[1:100])
corpus = []
for col in summarize_text:
    word_list = col.split(" ")
    corpus.append(word_list)
#show first value
corpus[0:1]


['Do androids dream of electric cars     Nothing if not discreet  Hey babe     We will use a catapult amp air mattresses to land on the roof, then base jump off   Nickel is our biggest concern for scaling lithium-ion cell production.', 'Thats why we are shifting standard range cars to an iron cathode.', 'Plenty of iron and lithium   Absolutely   Fremont shut down for two days parts shortages amp restarted yesterday  This is not about winning anyone over.', 'We are production-limited.', 'The reason we are making our own cells is to supplement max production of suppliers.', 'Even moving at full speed, they cannot build enough cells.', 'I hope they do It would be awesome    Prices in the US of all goods dont include VAT, but its usually included in other countries.', 'Then theres import duty amp transport costs.', 'The intent is that revenue to Tesla is roughly similar in all countries, so taxes, tariffs amp transport make up the difference.', '3mm of full hard 301 or 304 Stainless is bul

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/connormcsweeney/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


2
3
4
Indexes of top ranked_sentence order are  [(0.02057599822894371, 'Seems likely   Were upgrading all NNs to surround video, using subnets on focal areas vs equal compute on all uncropped pixels amp many other things, so more time needed to write amp validate software.'), (0.02055014814003248, 'Just a Surface Detail  I hope we dont go bankrupt building them Excession      Super cool   We have too much product complexity already   Tunnels    Hopefully, later this year   Later this year   It is still available off menu, but I dont think the range, in many drive conditions, yet meets the Tesla standard of excellence  The Tesla Solar Roof is slightly hydrophobic, so water, snow or ice slide off easily  Final design is looking Was just in the studio   Most of Earth by end of year, all by next year, then its about densifying coverage.'), (0.02048016612077398, 'The intent is that revenue to Tesla is roughly similar in all countries, so taxes, tariffs amp transport make up the difference.'

[['Seems',
  'likely',
  '',
  '',
  'Were',
  'upgrading',
  'all',
  'NNs',
  'to',
  'surround',
  'video,',
  'using',
  'subnets',
  'on',
  'focal',
  'areas',
  'vs',
  'equal',
  'compute',
  'on',
  'all',
  'uncropped',
  'pixels',
  'amp',
  'many',
  'other',
  'things,',
  'so',
  'more',
  'time',
  'needed',
  'to',
  'write',
  'amp',
  'validate',
  'software.']]

In [6]:
model = Word2Vec(corpus, min_count=1)
def Themer(ThemeList):
    counterlength = len(ThemeList)
    ThemeMeaningList = []
    while counterlength > 0:
        for index, item in enumerate(ThemeList):
            try:
                MeaningModel = model.wv.most_similar(item, topn=50000)
                #PotentialThemes.append(test2[0:5])
                MeaningThemeList = []
                for MeaningIndex, MeaningModelValue in enumerate(MeaningModel):
                    ThemeName = item 
                    Meaning = MeaningModel[MeaningIndex][0]
                    WordVecNumber = MeaningModel[MeaningIndex][1]
                    TempMeaningList = [ThemeName, Meaning, WordVecNumber]
                    #print(TempMeaningList)
                    ThemeMeaningList.append(TempMeaningList)
                ThemeList.pop(index)
                counterlength = counterlength - 1
                #ThemeMeaningList.append(MeaningThemeList)
            except:
                ThemeList.pop(index)
                counterlength = counterlength - 1
    ThemeMeaningList = ThemeMeaningList
    return ThemeMeaningList

In [7]:
with open('words.txt') as f:
    lines = f.read().splitlines()
wordlist = lines

ThemeListOfList = Themer(wordlist)


In [9]:

print(ThemeListOfList)
df = pd.DataFrame(ThemeListOfList)
df = df.sort_values(by=[2],ascending=False)
df.to_csv('PedoBearKingThailandJustice.csv', index=False) 

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
print(dfValaSentences.tolist())

In [ ]:
model.wv.most_similar('car', topn=50000)

In [ ]:
    ThemeList = ["Manager","Money","Security","risk","IAVA"]
    counterlength = len(ThemeList)
    ThemeMeaningList = []
    while counterlength > 0:
        for index, item in enumerate(ThemeList):
            try:
                MeaningModel = model.wv.most_similar(item, topn=50000)
                #PotentialThemes.append(test2[0:5])
                MeaningThemeList = []
                for MeaningIndex, MeaningModelValue in enumerate(MeaningModel):
                    ThemeName = item 
                    Meaning = MeaningModel[MeaningIndex][0]
                    WordVecNumber = MeaningModel[MeaningIndex][1]
                    TempMeaningList = [ThemeName, Meaning, WordVecNumber]
                    print(TempMeaningList)
                    ThemeMeaningList.append(TempMeaningList)
                ThemeList.pop(index)
                counterlength = counterlength - 1
                #ThemeMeaningList.append(MeaningThemeList)
            except:
                ThemeList.pop(index)
                counterlength = counterlength - 1

In [ ]:
print(combi)

In [ ]:
print(text)
originaltext = text
for item in text:
    item = tknzr.tokenize(item)

In [ ]:
print(text)

In [3]:
print(text)

['Do androids dream of electric cars', '  ', ' Nothing if not discreet', ' Hey babe ', '   We will use a catapult amp air mattresses to land on the roof, then base jump off', '  Nickel is our biggest concern for scaling lithium-ion cell production. Thats why we are shifting standard range cars to an iron cathode. Plenty of iron and lithium', '  Absolutely', '  Fremont shut down for two days parts shortages amp restarted yesterday', ' This is not about winning anyone over. We are production-limited. The reason we are making our own cells is to supplement max production of suppliers. Even moving at full speed, they cannot build enough cells.', ' I hope they do It would be awesome ', '  Prices in the US of all goods dont include VAT, but its usually included in other countries.   Then theres import duty amp transport costs.  The intent is that revenue to Tesla is roughly similar in all countries, so taxes, tariffs amp transport make up the difference.', ' 3mm of full hard 301 or 304 Stain